In [ ]:
!pip install keras

In [ ]:
%pylab inline

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import cross_val_score, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LassoCV, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import seaborn

In [ ]:
seaborn.set_context('talk')
seaborn.set_style('white')

# Load Dataset

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/danybol/gft_ml_example/master/worked_example/data.csv")

In [ ]:
data.head()

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
_ = plot(data.date, data.y)

In [ ]:
_ = plot(data.date[0:365], data.y[0:365])

#### Features to notice

- Seasonality
- Trend, possibly non-linear?

In [ ]:
y = data['y'].values
X = data.drop(['y', 'date'], axis=1).values

## Split into train and test

In [ ]:
def train_test_split(X, y):
    N = X.shape[0]
    split_size = int(N/5)
    split = int(N - 2*split_size)
    train_X = X[:split]
    train_y = y[:split]
    val_X = X[split:split+split_size]
    val_y = y[split:split+split_size]
    test_X = X[split+split_size:]
    test_y = y[split+split_size:]
    
    return train_X, train_y, val_X, val_y, test_X, test_y

In [ ]:
train_X, train_y, val_X, val_y, test_X, test_y = train_test_split(X, y)

### Set up framework for testing models

In [ ]:
def test_model(model, params, train_X, train_y, test_X, test_y):
    scorer = make_scorer(mean_squared_error, greater_is_better=False) # Use mean squared error as score
    gs = GridSearchCV(model, params, scoring=scorer) # Cross-validation to pick best hyperparameter
    gs.fit(train_X, train_y)
    gs.best_estimator_.fit(train_X, train_y)
    train_pred = gs.best_estimator_.predict(train_X) # Make prediction on training set
    test_pred = gs.best_estimator_.predict(test_X) # Make prediction on test set
    print("Training MSE: ", mean_squared_error(train_y, train_pred))
    print("Test MSE: ", mean_squared_error(test_y, test_pred))
    return gs.best_estimator_

#### Scikit-learn pipelines are a good way of keeping code clean. They let you easily swap out models and preprocessing steps

In [ ]:
model = Pipeline([('features', None),
                  ('preprocess', None),
                  ('model', None)])


### Try out different models

#### Lasso with no other features

In [ ]:
params = dict(model=[Lasso()], model__alpha=np.logspace(-2, 3), features=[None], preprocess=[None])
simple_model = test_model(model, params, train_X, train_y, val_X, val_y)
val_pred = simple_model.predict(val_X)

In [ ]:
plot(val_y)
plot(val_pred)
legend(['Truth','Prediction'])

#### Didn't pick up the trend (because lower than truth), didn't pick up seasonality. Looks like it does pick up some of the pattern in the 'noise' (possibly)

#### Let's do some feature engineering: add one-hot-encoded day of week and day of month

In [ ]:
week_one_hot = pd.get_dummies(data.date.dt.strftime("%a"))
print(week_one_hot.head())
month_one_hot = pd.get_dummies(data.date.dt.strftime("%b"))
print(month_one_hot.head())
time_in_days = (data.date - dt.datetime(2010, 1, 1)).dt.total_seconds() / 3600 / 24
df_dates = pd.concat([data, week_one_hot, month_one_hot], axis=1)
df_dates['t'] = time_in_days
data = df_dates
X = data.drop(['y', 'date'], axis=1).values
train_X, train_y, val_X, val_y, test_X, test_y = train_test_split(X, y)

In [ ]:
params = dict(model=[Lasso()], model__alpha=np.logspace(-2, 3), features=[None], preprocess=[None])
week_month_model = test_model(model, params, train_X, train_y, val_X, val_y)
val_pred = week_month_model.predict(val_X)

In [ ]:
plot(val_y)
plot(val_pred)

#### Captures the seasonality, but missing trend

#### Let's try standardising the input variables to see if that helps

In [ ]:
params = dict(model=[Lasso()], model__alpha=np.logspace(-2, 6), features=[None], preprocess=[StandardScaler()])
standardised_model = test_model(model, params, train_X, train_y, val_X, val_y)
val_pred = standardised_model.predict(val_X)

In [ ]:
plot(val_y)
plot(val_pred)

#### Let's try and capture the trend by adding nonlinear combinations of variables (e.g. $X_{0}^2$, $X_{0}$*$X_{1}$, etc)

In [ ]:
params = dict(model=[Lasso()], model__alpha=np.logspace(-1, 3, num=10), preprocess=[StandardScaler()], features=[PolynomialFeatures()])
trend_model = test_model(model, params, train_X, train_y, val_X, val_y)
val_pred = trend_model.predict(val_X)

In [ ]:
plot(val_y)
plot(val_pred)

#### Better MSE and capturing the trend slightly better. Perhaps there are higher order terms?

### Try deep learning to avoid manual feature engineering

#### Let's see if Deep learning can do better, just from the patterns in the target variable

In [ ]:
pd.plotting.autocorrelation_plot(y)

In [ ]:

from keras import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator

seq_len = 100 # Length of autocorrelation from plot above

N = train_y.shape[0]
split = int(N - N/4)
sc = StandardScaler() # Scale output variable to make convergence faster. Remember to reverse this at the end!
new_train_y = train_y[:split]
new_train_y = sc.fit_transform(new_train_y[:, np.newaxis])[:,0]
sub_val_y = train_y[split:]
sub_val_y = sc.transform(sub_val_y[:, np.newaxis])[:, 0]


#### Convert target variable into a format suitable for LSTM

In [ ]:
train_data = TimeseriesGenerator(new_train_y[:, np.newaxis], new_train_y,
                               length=seq_len, sampling_rate=1,
                               batch_size=10)

sub_val_data = TimeseriesGenerator(sub_val_y[:, np.newaxis], sub_val_y,
                               length=seq_len, sampling_rate=1,
                               batch_size=10)

val_data = TimeseriesGenerator(val_y[:, np.newaxis], val_y,
                               length=seq_len, sampling_rate=1,
                               batch_size=10)


#### Very simple LSTM. Can be improved by adding additional LSTM cells and dropout layers

In [ ]:
neurons = 20
n_input=seq_len
n_features = 1
model = Sequential()
model.add(LSTM(neurons, input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')


history = model.fit_generator(train_data, epochs=10, validation_data=sub_val_data, verbose=1)



In [ ]:
preds = []
fdata = sub_val_y[-seq_len:]
for i in range(val_y.shape[0]):
    pred = model.predict(fdata.reshape((1, seq_len, 1)))
    preds.append(pred[0][0])
    fdata = np.roll(fdata, -1)
    fdata[-1] = pred

val_pred = np.asarray(preds)
val_pred = sc.inverse_transform(val_pred)
mean_squared_error(val_y, val_pred)

In [ ]:
plot(val_y)
plot(val_pred)

_ = legend(['actual', 'prediction'])

#### Better than the naive model, but not really picking up seasonality or trend. May need more layers and more training

# Test the best model on the test dataset

#### Finally, test the model we have chosen on an unseen set of data. Hopefully the model will generalise well

In [ ]:
test_pred = trend_model.predict(test_X)
mean_squared_error(test_y, test_pred)

In [ ]:
plot(test_y)
plot(test_pred)